<h3> Leetcode Hard Problem | Complex SQL 7 | Trips and Users </h3>

In [0]:
%sql
Create table  Trips (id int, client_id int, driver_id int, city_id int, status varchar(50), request_at varchar(50));
Create table Users (users_id int, banned varchar(50), role varchar(50));
Truncate table Trips;
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('1', '1', '10', '1', 'completed', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('3', '3', '12', '6', 'completed', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('4', '4', '13', '6', 'cancelled_by_client', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('5', '1', '10', '1', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('6', '2', '11', '6', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('7', '3', '12', '6', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('8', '2', '12', '12', 'completed', '2013-10-03');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('9', '3', '10', '12', 'completed', '2013-10-03');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03');
Truncate table Users;
insert into Users (users_id, banned, role) values ('1', 'No', 'client');
insert into Users (users_id, banned, role) values ('2', 'Yes', 'client');
insert into Users (users_id, banned, role) values ('3', 'No', 'client');
insert into Users (users_id, banned, role) values ('4', 'No', 'client');
insert into Users (users_id, banned, role) values ('10', 'No', 'driver');
insert into Users (users_id, banned, role) values ('11', 'No', 'driver');
insert into Users (users_id, banned, role) values ('12', 'No', 'driver');
insert into Users (users_id, banned, role) values ('13', 'No', 'driver');


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from Trips

id,client_id,driver_id,city_id,status,request_at
4,4,13,6,cancelled_by_client,2013-10-01
2,2,11,1,cancelled_by_driver,2013-10-01
10,4,13,12,cancelled_by_driver,2013-10-03
7,3,12,6,completed,2013-10-02
1,1,10,1,completed,2013-10-01
6,2,11,6,completed,2013-10-02
3,3,12,6,completed,2013-10-01
5,1,10,1,completed,2013-10-02
8,2,12,12,completed,2013-10-03
9,3,10,12,completed,2013-10-03


In [0]:
%sql
select * from Users

users_id,banned,role
2,Yes,client
1,No,client
3,No,client
4,No,client
10,No,driver
11,No,driver
12,No,driver
13,No,driver


<h3> Solution in sql(hive) </h3>

In [0]:
%sql

select t.request_at,sum(case when t.status<>'completed' then 1 else 0 end) as cancel_trip_count,
count(*) as total_trip,
sum(case when t.status<>'completed' then 1 else 0 end)*100/count(*) as cancel_percentage
 from trips t
inner join users c1 on c1.users_id=t.client_id
inner join users d1 on d1.users_id=t.driver_id
where c1.banned<>'Yes' and d1.banned<>'Yes'
group by t.request_at

request_at,cancel_trip_count,total_trip,cancel_percentage
2013-10-01,1,3,33.333333333333336
2013-10-03,1,2,50.0
2013-10-02,0,2,0.0


<h3> Solution in pyspark </h3>

In [0]:
from pyspark.sql.functions import col,count

In [0]:
df_u=spark.sql("select * from users")
df_t=spark.sql("select * from trips")
df_t.show()
df_u.show()

+---+---------+---------+-------+-------------------+----------+
| id|client_id|driver_id|city_id|             status|request_at|
+---+---------+---------+-------+-------------------+----------+
|  4|        4|       13|      6|cancelled_by_client|2013-10-01|
|  2|        2|       11|      1|cancelled_by_driver|2013-10-01|
| 10|        4|       13|     12|cancelled_by_driver|2013-10-03|
|  7|        3|       12|      6|          completed|2013-10-02|
|  1|        1|       10|      1|          completed|2013-10-01|
|  6|        2|       11|      6|          completed|2013-10-02|
|  3|        3|       12|      6|          completed|2013-10-01|
|  5|        1|       10|      1|          completed|2013-10-02|
|  8|        2|       12|     12|          completed|2013-10-03|
|  9|        3|       10|     12|          completed|2013-10-03|
+---+---------+---------+-------+-------------------+----------+

+--------+------+------+
|users_id|banned|  role|
+--------+------+------+
|       2|   Y

In [0]:
df1=df_t.join(df_u,df_u.users_id==df_t.client_id,"inner").filter(col("banned")=='No').select("driver_id","status","request_at")
df2=df1.join(df_u,df_u.users_id==df1.driver_id,"inner").filter(col("banned")=='No').select("status","request_at")
df_tot=df2.groupBy("request_at").agg(count("request_at").alias("total_trips"))
df_can=df2.filter(col("status")!='completed').groupBy("request_at").agg(count("request_at").alias("cancle_trips_count"))
df_fin=df_tot.join(df_can, "request_at", "left").fillna({"cancle_trips_count":0})
df_final=df_fin.withColumn("cancel_percenateg",df_fin.cancle_trips_count*100/df_fin.total_trips)
df_final.orderBy("request_at").select("request_at","cancle_trips_count","total_trips","cancel_percenateg").show()

+----------+------------------+-----------+------------------+
|request_at|cancle_trips_count|total_trips| cancel_percenateg|
+----------+------------------+-----------+------------------+
|2013-10-01|                 1|          3|33.333333333333336|
|2013-10-02|                 0|          2|               0.0|
|2013-10-03|                 1|          2|              50.0|
+----------+------------------+-----------+------------------+

